## Getting Started with Prompt Management and Flows for Amazon Bedrock

This example shows you how to get started with Prompt Management and Prompt Flows in Amazon Bedrock.

[Amazon Bedrock Prompt Management](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management.html) streamlines the creation, evaluation, deployment, and sharing of prompts in the Amazon Bedrock console and via APIs in the SDK. This feature helps developers and business users obtain the best responses from foundation models for their specific use cases.

[Amazon Bedrock Prompt Flows](https://docs.aws.amazon.com/bedrock/latest/userguide/flows.html) allows you to easily link multiple foundation models (FMs), prompts, and other AWS services, reducing development time and effort. It introduces a visual builder in the Amazon Bedrock console and a new set of APIs in the SDK, that simplifies the creation of complex generative AI workflows.

Let's start by making sure we have the lastest version of the Amazon Bedrock SDK, importing the libraries, and setting-up the client.

In [ ]:
# Only run this the first time...
!pip3 install boto3 botocore -qU

In [2]:
import boto3
from datetime import datetime
import json

Note the Prompt Management and Flows features are part of the Bedrock Agent SDK.

In [3]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = "us-east-1")

## Prompt Management

### Create and Manage Prompts

Let's create a sample prompt, in this case for a simple translation task.

In [4]:
response = bedrock_agent.create_prompt(
    name = f"MyTestPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description = "This is my test prompt for the customer service use case",
    variants = [
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.999,
                    "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        }

                    ],
                    "text": "You're a customer service agent for the ecommerce company Octank. Answer the following user query in a friendly and direct way: {{input}}"
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variant-001"
)
print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

{
  "ResponseMetadata": {
    "RequestId": "d87105eb-02e9-452c-9844-c04d62c4c9ea",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:43:37 GMT",
      "content-type": "application/json",
      "content-length": "1110",
      "connection": "keep-alive",
      "x-amzn-requestid": "d87105eb-02e9-452c-9844-c04d62c4c9ea",
      "x-amz-apigw-id": "cD607GoZoAMEWYQ=",
      "x-amzn-trace-id": "Root=1-66b171b8-2158a69d106cfef128bd4e05"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:prompt/ZVL0VXLDG3",
  "createdAt": "2024-08-06 00:43:36.869583+00:00",
  "defaultVariant": "variant-001",
  "description": "This is my test prompt for the customer service use case",
  "id": "ZVL0VXLDG3",
  "name": "MyTestPrompt-20240805-214333",
  "updatedAt": "2024-08-06 00:43:36.869583+00:00",
  "variants": [
    {
      "inferenceConfiguration": {
        "text": {
          "maxTokens": 3000,
          "temperature": 0.0,
          "topK": 250

Now that we have a draft prompt, we can create versions from it.

In [5]:
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "22647bc8-e2fe-4a3b-8b6e-c308ce501460",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:44:22 GMT",
      "content-type": "application/json",
      "content-length": "1035",
      "connection": "keep-alive",
      "x-amzn-requestid": "22647bc8-e2fe-4a3b-8b6e-c308ce501460",
      "x-amz-apigw-id": "cD679Gu2IAMEJ8A=",
      "x-amzn-trace-id": "Root=1-66b171e5-1a69bb2879e49870596f19b2"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:prompt/ZVL0VXLDG3:1",
  "createdAt": "2024-08-06 00:44:21.932257+00:00",
  "defaultVariant": "variant-001",
  "id": "ZVL0VXLDG3",
  "name": "MyTestPrompt-20240805-214333",
  "updatedAt": "2024-08-06 00:44:21.932257+00:00",
  "variants": [
    {
      "inferenceConfiguration": {
        "text": {
          "maxTokens": 3000,
          "temperature": 0.0,
          "topK": 250,
          "topP": 0.9990000128746033
        }
      },
      "modelId": 

Here we can see the list of prompts in our Prompt Library or catalog.

In [6]:
response = bedrock_agent.list_prompts(
    maxResults = 10
)
print(json.dumps(response["promptSummaries"], indent=2, default=str))

[
  {
    "arn": "arn:aws:bedrock:us-east-1:637423169504:prompt/ZVL0VXLDG3",
    "createdAt": "2024-08-06 00:43:36.869583+00:00",
    "description": "This is my test prompt for the customer service use case",
    "id": "ZVL0VXLDG3",
    "name": "MyTestPrompt-20240805-214333",
    "updatedAt": "2024-08-06 00:43:36.869583+00:00",
    "version": "DRAFT"
  }
]


We can also read the details of any of our prompts.

In [7]:
response = bedrock_agent.get_prompt(
    promptIdentifier = promptId,
    promptVersion = "1"
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "b85dae9d-c6df-4d93-8d66-796a739def41",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:44:29 GMT",
      "content-type": "application/json",
      "content-length": "1035",
      "connection": "keep-alive",
      "x-amzn-requestid": "b85dae9d-c6df-4d93-8d66-796a739def41",
      "x-amz-apigw-id": "cD69HEaoIAMECAA=",
      "x-amzn-trace-id": "Root=1-66b171ed-3f915d3f2dc1992f29ddd9be"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:prompt/ZVL0VXLDG3:1",
  "createdAt": "2024-08-06 00:44:21.932257+00:00",
  "defaultVariant": "variant-001",
  "id": "ZVL0VXLDG3",
  "name": "MyTestPrompt-20240805-214333",
  "updatedAt": "2024-08-06 00:44:21.932257+00:00",
  "variants": [
    {
      "inferenceConfiguration": {
        "text": {
          "maxTokens": 3000,
          "temperature": 0.0,
          "topK": 250,
          "topP": 0.9990000128746033
        }
      },
      "modelId": 

-------

## Prompt Flows

Now that we've learned how to create and manage prompts, we can continue exploring how to build generative AI applications logic by creating workflows. For this, we'll rely on Prompt Flows for Amazon Bedrock.

### Create and Manage Flows

Let's create a simple flow that will load a prompt from our catalog. Note you can also create more complex flows involving chaining of steps, and conditions for dynamically routing, but let's keep it simple for now.

***Pre-requisite: For using Flows you need to make sure you have the proper AWS IAM permissions in place. You can check details in the [How Prompt Flows for Amazon Bedrock works](https://docs.aws.amazon.com/bedrock/latest/userguide/flows-how-it-works.html) documentation.***

In [8]:
### REPLACE WITH YOUR AWS IAM ROLE WITH FLOWS FOR BEDROCK PERMISSIONS
flow_role = "arn:aws:iam::637423169504:role/service-role/AmazonBedrockExecutionRoleForFlows_4IE86KZXSCO"

In [9]:
response = bedrock_agent.create_flow(
    name = f"MyTestFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description = "This is my test flow for the customer service use case",
    executionRoleArn = flow_role,
    definition = {
      "nodes": [
          {
              "name": "StartNode",
              "type": "Input",
              "configuration": {
                  "input": {}
              },
              "outputs": [
                  {
                      "name": "document",
                      "type": "String"
                  }
              ],
          },
          {
            "name": "Prompt_1",
            "type": "Prompt",
            "configuration": {
              "prompt": {
                "sourceConfiguration": {
                  "resource": {
                      "promptArn": promptArn
                  }
                }
              }
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "input",
                "type": "String"
              }
            ],
            "outputs": [
              {
                "name": "modelCompletion",
                "type": "String"
              }
            ],
          },
          {
            "name": "EndNode",
            "type": "Output",
            "configuration": {
                "output": {}
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "document",
                "type": "String"
              }
            ],
          }
      ],
      "connections": [
          {
              "name": "Connection_1",
              "source": "StartNode",
              "target": "Prompt_1",
              "type": "Data",
              "configuration":{
                  "data": {
                      "sourceOutput": "document",
                      "targetInput": "input"
                  }
              }
          },
          {
              "name": "Connection_2",
              "source": "Prompt_1",
              "target": "EndNode",
              "type": "Data",
              "configuration": {
                  "data": {
                      "sourceOutput": "modelCompletion",
                      "targetInput": "document"
                  }
              }
          }
      ],
    }
)
print(json.dumps(response, indent=2, default=str))
flowId = response["id"]
flowArn = response["arn"]
flowName = response["name"]
print(f"Flow ID: {flowId}\nFlow ARN: {flowArn}\nFlow Name: {flowName}")

{
  "ResponseMetadata": {
    "RequestId": "9493798a-ec6b-45cc-b160-94e0ff5c691a",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:47:42 GMT",
      "content-type": "application/json",
      "content-length": "1327",
      "connection": "keep-alive",
      "x-amzn-requestid": "9493798a-ec6b-45cc-b160-94e0ff5c691a",
      "x-amz-apigw-id": "cD7bVE_2IAMEuGQ=",
      "x-amzn-trace-id": "Root=1-66b172ae-57c5efaf533f5e431b13477a"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
  "createdAt": "2024-08-06 00:47:42.712717+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

Now that we have our first flow, we can prepare it. This basically builds and validates our flow.

In [10]:
response = bedrock_agent.prepare_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "160459c9-6283-43a7-9cde-c7bfba87df88",
    "HTTPStatusCode": 202,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:47:51 GMT",
      "content-type": "application/json",
      "content-length": "40",
      "connection": "keep-alive",
      "x-amzn-requestid": "160459c9-6283-43a7-9cde-c7bfba87df88",
      "x-amz-apigw-id": "cD7ctHtmoAMEItQ=",
      "x-amzn-trace-id": "Root=1-66b172b7-154fcb813e8cec3813cb937d"
    },
    "RetryAttempts": 0
  },
  "id": "O2VPKQ9ROW",
  "status": "Preparing"
}


In [11]:
response = bedrock_agent.get_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "e7e133ba-9e20-4343-a1cb-b699d60db1f1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:47:54 GMT",
      "content-type": "application/json",
      "content-length": "1344",
      "connection": "keep-alive",
      "x-amzn-requestid": "e7e133ba-9e20-4343-a1cb-b699d60db1f1",
      "x-amz-apigw-id": "cD7dHGH_IAMEnnA=",
      "x-amzn-trace-id": "Root=1-66b172ba-203069b90269c1120f0c9385"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
  "createdAt": "2024-08-06 00:47:42.712717+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

We can also list all the flows in our account.

In [12]:
response = bedrock_agent.list_flows(
    maxResults=10,
)
print(json.dumps(response["flowSummaries"], indent=2, default=str))

[
  {
    "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
    "createdAt": "2024-08-06 00:47:42.712717+00:00",
    "description": "This is my test flow for the customer service use case",
    "id": "O2VPKQ9ROW",
    "name": "MyTestFlow-20240805-214741",
    "status": "Prepared",
    "updatedAt": "2024-08-06 00:47:42.712717+00:00",
    "version": "DRAFT"
  },
  {
    "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/SZBYJ2UQPI",
    "createdAt": "2024-08-06 00:46:19.618208+00:00",
    "id": "SZBYJ2UQPI",
    "name": "test",
    "status": "NotPrepared",
    "updatedAt": "2024-08-06 00:46:19.618208+00:00",
    "version": "DRAFT"
  }
]


Let's create a version from our draft flow. Note flow versions are read-only, meaning these cannot be modified once created as they're intended for using in production. If you need to make changes to a flow you can update your draft.

In [13]:
response = bedrock_agent.create_flow_version(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "6a473a23-0a6c-4bc4-942a-63d1695b679d",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:48:02 GMT",
      "content-type": "application/json",
      "content-length": "1207",
      "connection": "keep-alive",
      "x-amzn-requestid": "6a473a23-0a6c-4bc4-942a-63d1695b679d",
      "x-amz-apigw-id": "cD7eWGxjoAMESrg=",
      "x-amzn-trace-id": "Root=1-66b172c1-25b112756404958c04471b3b"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
  "createdAt": "2024-08-06 00:48:01.973122+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

We can also create flow alises, so that we can point our application front-ends and any other integrations to these. This allows creating new versions without impacting our service.

In [14]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier = flowId,
    name = flowName,
    description = "Alias for my test flow in the customer service use case",
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
print(json.dumps(response, indent=2, default=str))
flowAliasId = response['id']

{
  "ResponseMetadata": {
    "RequestId": "5b159187-575b-418a-a084-6121458af55f",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:48:08 GMT",
      "content-type": "application/json",
      "content-length": "364",
      "connection": "keep-alive",
      "x-amzn-requestid": "5b159187-575b-418a-a084-6121458af55f",
      "x-amz-apigw-id": "cD7fWFdeIAMESug=",
      "x-amzn-trace-id": "Root=1-66b172c8-332f870e57e75f6e0b665cbb"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW/alias/0RSWXUMVLX",
  "createdAt": "2024-08-06 00:48:08.433956+00:00",
  "description": "Alias for my test flow in the customer service use case",
  "flowId": "O2VPKQ9ROW",
  "id": "0RSWXUMVLX",
  "name": "MyTestFlow-20240805-214741",
  "routingConfiguration": [
    {
      "flowVersion": "1"
    }
  ],
  "updatedAt": "2024-08-06 00:48:08.433956+00:00"
}


In [15]:
import json
response = bedrock_agent.list_flow_versions(
    flowIdentifier = flowId,
    maxResults = 10
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "4a8704f9-715b-4dbf-a3a0-4499b8ff18a4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:48:11 GMT",
      "content-type": "application/json",
      "content-length": "354",
      "connection": "keep-alive",
      "x-amzn-requestid": "4a8704f9-715b-4dbf-a3a0-4499b8ff18a4",
      "x-amz-apigw-id": "cD7f2E2_IAMEWQA=",
      "x-amzn-trace-id": "Root=1-66b172cb-403bd00a4c4afdd7451d005d"
    },
    "RetryAttempts": 0
  },
  "flowVersionSummaries": [
    {
      "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
      "createdAt": "2024-08-06 00:47:42.712717+00:00",
      "id": "O2VPKQ9ROW",
      "status": "Prepared",
      "version": "DRAFT"
    },
    {
      "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
      "createdAt": "2024-08-06 00:48:01.973122+00:00",
      "id": "O2VPKQ9ROW",
      "status": "Prepared",
      "version": "1"
    }
  ]
}


In [16]:
response = bedrock_agent.get_flow_version(
    flowIdentifier = flowId,
    flowVersion = '1'
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "32cf8a84-8373-402d-9b7a-7a4308422acd",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:48:15 GMT",
      "content-type": "application/json",
      "content-length": "1207",
      "connection": "keep-alive",
      "x-amzn-requestid": "32cf8a84-8373-402d-9b7a-7a4308422acd",
      "x-amz-apigw-id": "cD7gaF4WIAMEMDA=",
      "x-amzn-trace-id": "Root=1-66b172cf-2501f1bd1b3d95521435bada"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW",
  "createdAt": "2024-08-06 00:48:01.973122+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

We can also update a given alias assigned to a flow, for e.g. pointing to another version if required.

In [17]:
response = bedrock_agent.update_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId,
    name = flowName,
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
flowAliasId = response["id"]
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "3fe82d6b-bd29-4e1f-92a7-d8e9ff0908e0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:48:19 GMT",
      "content-type": "application/json",
      "content-length": "247",
      "connection": "keep-alive",
      "x-amzn-requestid": "3fe82d6b-bd29-4e1f-92a7-d8e9ff0908e0",
      "x-amz-apigw-id": "cD7hBHoDIAMEizg=",
      "x-amzn-trace-id": "Root=1-66b172d3-463a6df1219cfb58196594fb"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW/alias/0RSWXUMVLX",
  "flowId": "O2VPKQ9ROW",
  "id": "0RSWXUMVLX",
  "name": "MyTestFlow-20240805-214741",
  "routingConfiguration": [
    {
      "flowVersion": "1"
    }
  ],
  "updatedAt": "2024-08-06 00:48:08.433956+00:00"
}


In [18]:
response = bedrock_agent.get_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "eb87e3ae-5042-4356-b85a-60b7784f12d4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 06 Aug 2024 00:48:24 GMT",
      "content-type": "application/json",
      "content-length": "292",
      "connection": "keep-alive",
      "x-amzn-requestid": "eb87e3ae-5042-4356-b85a-60b7784f12d4",
      "x-amz-apigw-id": "cD7hzF75IAMEH5Q=",
      "x-amzn-trace-id": "Root=1-66b172d8-49d80c9108b0909525d3dd00"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-east-1:637423169504:flow/O2VPKQ9ROW/alias/0RSWXUMVLX",
  "createdAt": "2024-08-06 00:48:08.433956+00:00",
  "flowId": "O2VPKQ9ROW",
  "id": "0RSWXUMVLX",
  "name": "MyTestFlow-20240805-214741",
  "routingConfiguration": [
    {
      "flowVersion": "1"
    }
  ],
  "updatedAt": "2024-08-06 00:48:08.433956+00:00"
}


### Invoke a Flow

Now that we have learned how to create and manage flows, we can test these with invocations.

Note for this we'll rely on the Bedrock Agent Runtime SDK.

You can invoke flows from any application front-end or your own systems as required. It effectively exposes all the logic of your flow through an Agent Endpoint API.

In [19]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = 'us-east-1')

In [23]:
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier = flowId,
    flowAliasIdentifier = flowAliasId,
    inputs = [
        { 
            "content": { 
                "document": "Hi, I need help with my order!"
            },
            "nodeName": "StartNode",
            "nodeOutputName": "document"
        }
    ]
)
event_stream = response["responseStream"]
for event in event_stream:
    print(json.dumps(event, indent=2, ensure_ascii=False))

EventStreamError: An error occurred (accessDeniedException) when calling the InvokeFlow operation: Access denied. Check your request permissions and retry the request.

---

### Cleaning-up Resources (optional)

Before leaving, here's how to delete the resources that we've created.

In [ ]:
response = bedrock_agent.delete_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_flow_version(
    flowIdentifier = flowId,
    flowVersion = '1'
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_prompt(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

-------